heartbeat thump?


In [ ]:
var fs = require('fs');
var path = require('path');

var CORE = path.resolve(path.join(__dirname, '../Core'));
var NODE = fs.existsSync('/usr/local/bin/node') ? '/usr/local/bin/node' : '/usr/bin/node';

function thump(thump, parameters, exit) {
    var out = fs.openSync('/tmp/myjob.log', 'a');
    var err = fs.openSync('/tmp/myjob.log', 'a');
    console.log('executing ' + NODE + ' - ' + CORE + ' - ' + thump + ' - ' + JSON.stringify(parameters));
    require('child_process').spawn(NODE, [
        '-e',
        'require(' + JSON.stringify(CORE) + ').import("' + thump + '")(' + (parameters ? ('...' + JSON.stringify(parameters || [])) : '') + ').catch(e => console.log(e)).then(() => process.exit())'
    ], {
        detached: true,
        stdio: [ 'ignore', out, err ]
    }).unref();
    if(exit !== false) {
        if(typeof $$ !== 'undefined') {
            return $$.done();
        }
        process.exit(0);
    }
}
module.exports = thump;


run todays heartbeat items?


In [17]:
var importer = require('../Core');
var {JSDOM} = require('jsdom');
var {
    getOauthClient,
    getDaysEvents,
    thump
} = importer.import([
    'days events',
    'import google calendar api',
    'heartbeat thump'
]);

var options = {
    calendarId: 'aws'
};

// test Google calendar API?
function runTodaysHeartbeat(calendar, exit) {
    if(calendar) {
        options.calendarId = calendar;
    }
    return (typeof options.auth === 'undefined'
           ? getOauthClient(options)
           : Promise.resolve([]))
        .then(() => getDaysEvents(new Date(), options))
        .then(r => {
            const heartbeat = r.filter(r => r.event.summary === 'heartbeat'
                                       || r.event.summary === 'todays heartbeat items');
        
            try {
                var dom = new JSDOM('<body>' + (heartbeat[0].event.description || '').replace(/<br\/?>/igm, '\n') + '</body>');
                const desc = dom.window.document.body.textContent.split('\n');
                desc.forEach(c => {
                    if(c.trim().length > 0) {
                        const parameters = (/[\{].*[\}]|[\[].*[\]]|(['"]).*\1/igm).exec(c) || ['""'];
                        thump(c.replace(parameters[0], ''), JSON.parse(parameters[0]), false);
                    }
                });
            } catch ( e ) {
                console.log(e);
            }
        
            if(exit !== false) {
                process.exit(0);
            }
        })
}
module.exports = runTodaysHeartbeat;

if(typeof $$ !== 'undefined') {
    $$.async();
    runTodaysHeartbeat(false, false)
        .then(r => $$.sendResult(r))
        .then(e => $$.sendError(e))
}


reading notebook /Users/briancullinan/jupytangular2/Utilities/heartbeat.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/heartbeat.ipynb[0]
{ timeMin: '2018-01-04T07:00:00-00:00',
  timeMax: '2018-01-05T07:00:00-00:00' }
executing /usr/local/bin/node - /Users/briancullinan/jupytangular2/Core - screenshot all sessions
executing /usr/local/bin/node - /Users/briancullinan/jupytangular2/Core - run todays calendar events 
executing /usr/local/bin/node - /Users/briancullinan/jupytangular2/Core - github updates 
